In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

In [2]:
import pandas as pd
import numpy as np
from konlpy.tag import Okt
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
import keras
import sys
import pickle
import re
import time
import warnings
import konlpy
from konlpy.tag import Kkma
kkma = Kkma(max_heap_size= 1024 * 6)

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.pipeline import Pipeline
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold

from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import train_test_split, cross_val_score,learning_curve
from sklearn import metrics
from sklearn.metrics import classification_report, f1_score, accuracy_score, confusion_matrix

from sklearn.externals import joblib

!git clone https://github.com/e9t/nsmc.git

train = pd.read_csv("nsmc/ratings_train.txt", sep='\t')
test = pd.read_csv("nsmc/ratings_test.txt", sep='\t')

print(train.shape)
print(test.shape)

fatal: destination path 'nsmc' already exists and is not an empty directory.


(150000, 3)
(50000, 3)


In [3]:
okt=Okt()
iter=0
def tokenize_data(sentence:str, ner=0)->list:
    x_tok=[]
    if ner == 1:
        try:
            parsed = recognizer(sentence)
        except:
            return ["없음"]
        
        c=parsed[0].getEntities()
        dic={}
        for i in  c:
            dic[i.getSurface()]=i.getFineLabel()
    try:
        a=okt.pos(sentence)
        b=np.array(a)
        c=(b[:,1] =='Noun')|(b[:,1] =='Adjective')|(b[:,1] =='KoreanParticle')|(b[:,1] =='Verb')|(b[:,1] =='Adverb')|(b[:,1] =='Exclamation')|(b[:,1] =='Conjunction')
        d=b[c,0]
        for word in d:
            if ner==1:
                if dic.get(word):
                    x_tok.append(dic[word])
                else:
                    x_tok.append(word)
            else:
                x_tok.append(word)
    except:
        return ["없음"]       
    return x_tok

In [4]:
def get_nouns(text:str) -> list:
    """ transform text to list
    : text : one news
    """
    nouns = kkma.nouns(text)
    return [n for n in nouns]

In [5]:
train=train.dropna()
train=train.drop_duplicates()
test=test.dropna()
test=test.drop_duplicates()

traindata = train[['document','label']]
testdata = test[['document','label']]

In [6]:
train_x=traindata.document
train_y=traindata.label

In [7]:
test_x=testdata.document
test_y=testdata.label

## 각 모델 생성 및 결과 도출 및 평가

In [46]:
pipeline = Pipeline([
    ('bow', CountVectorizer(analyzer=tokenize_data)),  # bow로 word2vec
    ('tfidf', TfidfTransformer(use_idf=False)),  # weighted TF-IDF scores
    ('classifier', MultinomialNB()),  # 나이브베이즈 분류기
])

In [47]:
%time model_nb = pipeline.fit(train_x, train_y)

Wall time: 4min 13s


In [48]:
predictions_nb=model_nb.predict(test_x)

### 나이브 베이즈 모델 결과

In [49]:
print (classification_report(test_y, predictions_nb))

              precision    recall  f1-score   support

           0       0.84      0.86      0.85     24826
           1       0.85      0.84      0.85     25171

    accuracy                           0.85     49997
   macro avg       0.85      0.85      0.85     49997
weighted avg       0.85      0.85      0.85     49997



In [50]:
joblib.dump(model_nb, 'naive.joblib')

['naive.joblib']

In [68]:
pipeline_logistic=Pipeline([
            ('bow', CountVectorizer(tokenizer=tokenize_data)),
            ('tfidf', TfidfTransformer(use_idf=False)),
            ('classifier', LogisticRegression(solver='newton-cg'))
            ])

In [69]:
%time model_logistic=pipeline_logistic.fit(train_x, train_y)

Wall time: 4min 18s


In [70]:
predictions_logistic=model_logistic.predict(test_x)

### 로지스틱 리그레션 모델 결과

In [71]:
print (classification_report(test_y, predictions_logistic))

              precision    recall  f1-score   support

           0       0.83      0.85      0.84     24826
           1       0.85      0.82      0.84     25171

    accuracy                           0.84     49997
   macro avg       0.84      0.84      0.84     49997
weighted avg       0.84      0.84      0.84     49997



In [72]:
joblib.dump(model_logistic, 'logistic.joblib')

['logistic.joblib']

In [21]:
pipeline_randomforest=Pipeline([
            ('bow', CountVectorizer(tokenizer=tokenize_data, ngram_range=(1,2))),
            ('tfidf', TfidfTransformer(use_idf=False)),
            ('classifier', RandomForestClassifier(n_estimators=100, oob_score=True))
            ])

In [22]:
%time model_randomforest=pipeline_randomforest.fit(train_x, train_y)

Wall time: 2h 37min 52s


In [23]:
predictions_randomforest=model_randomforest.predict(test_x)

### 랜덤 포레스트 모델 결과

In [24]:
print (classification_report(test_y, predictions_randomforest))

              precision    recall  f1-score   support

           0       0.82      0.83      0.82     24826
           1       0.83      0.82      0.82     25171

    accuracy                           0.82     49997
   macro avg       0.82      0.82      0.82     49997
weighted avg       0.82      0.82      0.82     49997



In [25]:
joblib.dump(model_randomforest, 'randomforest.joblib')

['randomforest.joblib']

In [52]:
pipeline_gdboost=Pipeline([
            ('bow', CountVectorizer(tokenizer=tokenize_data)),
            ('tfidf', TfidfTransformer(use_idf=True)),
            ('classifier', GradientBoostingClassifier(random_state=0, max_depth=3, learning_rate=0.1))
            ])

In [53]:
%time model_gdboost=pipeline_gdboost.fit(train_x, train_y)

Wall time: 5min 7s


In [54]:
predictions_gdboost=model_gdboost.predict(test_x)

### 그라디언트부스트 모델 결과

In [55]:
print (classification_report(test_y, predictions_gdboost))

              precision    recall  f1-score   support

           0       0.65      0.88      0.75     24826
           1       0.82      0.53      0.64     25171

    accuracy                           0.70     49997
   macro avg       0.73      0.70      0.69     49997
weighted avg       0.73      0.70      0.69     49997



In [56]:
joblib.dump(model_gdboost, 'gdboost.joblib')

['gdboost.joblib']

In [77]:
#model_nb, model_logistic, model_gdboost, model_randomforest

In [78]:
def one_hot_result (result):
    ret=[]
    for i in result:
        onehot=[0,0]
        onehot[i]=1
        ret.append(onehot)
    return ret

## 평가 데이터 전처리 및  VOTE 결과 도출

In [116]:
leader_test = pd.read_csv("ko_data.csv",encoding='CP949')
leader_test= leader_test.rename(columns={'Sentence':'document'})

In [117]:
leader_test=leader_test['document']

In [119]:
result1=model_nb.predict(leader_test)


In [120]:
result2=model_logistic.predict(leader_test)

In [121]:
result3=model_gdboost.predict(leader_test)

In [122]:
result4=model_randomforest.predict(leader_test)

In [123]:
result1=one_hot_result(result1)

In [124]:
result2=one_hot_result(result2)

In [125]:
result3=one_hot_result(result3)

In [126]:
result4=one_hot_result(result4)

In [127]:
#tot_onehot=np.array(result1)+np.array(result2)+np.array(result3)+np.array(result4)

In [137]:
tot_onehot=np.array(result1)+np.array(result2)+np.array(result4)

In [138]:
tot_result=np.argmax(tot_onehot, axis=1)

In [141]:
import csv
f = open('sample_ensemble.csv','a', newline='')
wr = csv.writer(f)
wr.writerow(['Id','Predicted'])
for i in range(len(tot_result)):
  wr.writerow([i,tot_result[i]])
f.close()

In [133]:
tot_result[0]

1